In [1]:
from __future__ import division, print_function, absolute_import

seed_num = 45

import numpy as np
np.random.seed(seed_num)
import random
random.seed(seed_num)
import tensorflow as tf
tf.set_random_seed(seed_num)

import os
import shutil
import sys
import glob
import math
from functools import reduce
import itertools
import operator as op
from sklearn.utils import class_weight
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

dataset_folder = os.path.abspath("./individual_npzs_logmel/{0}/*.npz")
checkpoint_folder = "acrnn_topology3_es25"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

np.set_printoptions(suppress=True)

## Parameters

In [2]:
learning_rate = 0.00001
batch_size = 50
dropout = 0.1
recurrent_dropout = 0.9
final_dropout = 0.5
max_pool = 2
strides = 1
input_size = 60000
output_size = 4
epochs = 500
timesteps = 75

first_maps = 32
second_maps = 48
third_maps = 64

first_kernel = [3, 5, 1, first_maps]
second_kernel = [3, 5, first_maps, second_maps]
third_kernel = [3, 5, second_maps, third_maps]

hidden_layer = 256
dense_layer = 64

early_stop = 25

label_dictionary = {'ang': 0, 'hap': 1, 'neu': 2, 'sad': 3}
onehot_dictionary = {0: 'ang', 1: 'hap', 2:'neu', 3:'sad'}

## Helper methods

In [3]:
def batch_generator(data, labels):
    steps = math.ceil(data.shape[0] / batch_size)
    for batch_step in range(0, steps):
        start = batch_size * batch_step
        end = batch_size * (batch_step + 1)
        yield data[start:end], labels[start:end], batch_step

def compute_class_weights(labels):
    return np.ndarray.tolist(class_weight.compute_class_weight('balanced', np.unique(labels), labels))        

def build_encoded_array(emotion_label):
    initialized_array = [0. for key in label_dictionary]
    initialized_array[label_dictionary[emotion_label]] = 1.
    return initialized_array
        
def onehot_encode(label_minibatch):
    return [build_encoded_array(emotion_label) for emotion_label in label_minibatch]

def onehot_decode(onehot_labels):
    return [onehot_dictionary[label.index(1.0)] for label in onehot_labels]

def weighted_accuracy(y_pred, y_true):
    #Suma vsetkych spravne urcenych viet predelena vsetkymi vetami

    correct = 0
    all = 0
    for key in y_true:
        correct += y_pred[key]
        all += y_true[key]

    return correct/all


def unweighted_accuracy(y_pred, y_true):

    #Suma poctu spravnych viet lomeno vsetky vety pre kazdu emociu (triedu) predelena poctom tried

    correct = 0
    emotions = len(y_true)
    for key in y_true:
        correct += y_pred[key]/y_true[key]

    return correct/emotions

def validation_results(current_sess, data, labels):
    predictions = current_sess.run(correct_prediction, feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    coupled_validation = list(zip(predictions, onehot_decode(labels)))
    final_dict = {'ang':0, 'hap':0, 'neu':0, 'sad':0}
    overall_dict = {'ang':0, 'hap':0, 'neu':0, 'sad':0}

    for couple in coupled_validation:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    unweighted_acc = unweighted_accuracy(final_dict, overall_dict)
    return unweighted_acc

def compute_mean_std(data):
    dataset = []
    for key in data:
        dataset.extend(data[key]['M'][0])
        dataset.extend(data[key]['F'][0])
    
    dataset = np.asarray(dataset)
    data_to_norm = np.zeros([len(dataset), dataset[0].shape[0], dataset[0].shape[1]], dtype=np.float32)
    for spect_data in range(len(dataset)):
        data_to_norm[spect_data,:,:] = dataset[spect_data]
        
    dataset_mean = np.ndarray.astype(np.mean(data_to_norm, axis=(0,1,2), dtype='float64', keepdims=True), dtype='float32')
    dataset_std = np.ndarray.astype(np.std(data_to_norm, axis=(0,1,2), dtype='float64', keepdims=True), dtype='float32')

    return dataset_mean, dataset_std

## Load dataset: 4 sessions for training, 1 for test

In [4]:
def prepare_train_data(data, to_skip, mean, std):
    
    # load for train except validation and train session
    train_dataset = []
    train_labels = []
    for key in data:
        if key != to_skip:
            train_dataset.extend(data[key]['M'][0])
            train_dataset.extend(data[key]['F'][0])
            train_labels.extend(data[key]['M'][1])
            train_labels.extend(data[key]['F'][1])
    
    # transform to desired shapes
    train_dataset = np.asarray(train_dataset)
    train_data = np.zeros([len(train_dataset), train_dataset[0].shape[0], train_dataset[0].shape[1]], dtype=np.float32)
    for spect_data in range(len(train_dataset)):
        train_data[spect_data,:,:] = train_dataset[spect_data]
    
    other_dataset = np.concatenate((np.asarray(data[to_skip]['M'][0]), np.asarray(data[to_skip]['F'][0])))
    other_labs = np.concatenate((np.asarray(data[to_skip]['M'][1]), np.asarray(data[to_skip]['F'][1])))
    other_data = np.zeros([len(other_dataset), other_dataset[0].shape[0], other_dataset[0].shape[1]], dtype=np.float32)
    for spect_data in range(len(other_dataset)):
        other_data[spect_data,:,:] = other_dataset[spect_data]
        
    # z-norm the dataset
    train_data = (train_data - mean) / std
        
    # shuffle the dataset
    train_data, train_labels = shuffle(train_data, train_labels, random_state=seed_num)
    train_data = train_data.reshape((train_data.shape[0], input_size))

    # compute class_weights and onehot encode after
    class_weights = compute_class_weights(train_labels)
    train_labels = onehot_encode(train_labels)
    
    return train_data, train_labels, class_weights

In [ ]:
def prepare_validation_test_sets(data, gender, test_val_session, mean, std):
    
    test_dataset = data[test_val_session][gender][0]
    test_labels = data[test_val_session][gender][1]
    
    validation_dataset = []
    validation_labels = []
    
    if gender == 'M':
        validation_dataset = data[test_val_session]['F'][0]
        validation_labels = data[test_val_session]['F'][1]
    else:
        validation_dataset = data[test_val_session]['M'][0]
        validation_labels = data[test_val_session]['M'][1]
    
    # transform to desired shapes
    test_dataset = np.asarray(test_dataset)
    test_data = np.zeros([len(test_dataset), test_dataset[0].shape[0], test_dataset[0].shape[1]], dtype=np.float32)
    for spect_data in range(len(test_dataset)):
        test_data[spect_data,:,:] = test_dataset[spect_data]
    
    validation_dataset = np.asarray(validation_dataset)
    validation_data = np.zeros([len(validation_dataset), validation_dataset[0].shape[0], validation_dataset[0].shape[1]], dtype=np.float32)
    for spect_data in range(len(validation_dataset)):
        validation_data[spect_data,:,:] = validation_dataset[spect_data]
        
    # z-norm the dataset
    validation_data = (validation_data - mean) / std
    test_data = (test_data - mean) / std

    test_data = test_data.reshape((test_data.shape[0], input_size))
    validation_data = validation_data.reshape((validation_data.shape[0], input_size))

    # onehot encode labels
    test_labels = onehot_encode(test_labels)
    validation_labels = onehot_encode(validation_labels)
    
    return test_data, test_labels, validation_data, validation_labels

## Pipeline

In [ ]:
def conv2d(to_process, weights, biases, batchnorm_switch, strides=1):
    conv_out = tf.nn.conv2d(to_process, weights, strides=[1, strides, strides, 1], padding='SAME')
    bias_out = tf.nn.bias_add(conv_out, biases)
    batchnorm_out = tf.layers.batch_normalization(bias_out, training=batchnorm_switch)
    relu_out = tf.nn.leaky_relu(batchnorm_out, 0.01)
    return relu_out

def maxpool2d(to_pool, pool_size=2):
    maxpool_out = tf.nn.max_pool(to_pool, ksize=[1, pool_size, pool_size, 1], strides=[1, pool_size, pool_size, 1], padding='SAME')
    return maxpool_out

def attention(rnn_inputs, attention_size):

    hidden_size = rnn_inputs.shape[2].value

    W_omega = tf.Variable(tf.random_normal([hidden_size, attention_size], stddev=0.1))
    b_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    u_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))

    v = tf.sigmoid(tf.tensordot(rnn_inputs, W_omega, axes=1) + b_omega)
    vu = tf.tensordot(v, u_omega, axes=1) 
    alphas = tf.nn.softmax(vu)          

    output = tf.reduce_sum(rnn_inputs * tf.expand_dims(alphas, -1), 1)
    
    return output
   

def nn_pipeline_rnn(spectrogram, weights, biases, dropout_num, rnn_dropout, fully_connected_dropout, training_switch):
    reshaped_input = tf.reshape(spectrogram, shape=[-1, 200, 300, 1])

    first_layer_out = conv2d(reshaped_input, weights['first_layer_weights'], biases['first_layer_biases'], normalization_switch)
    first_drop_out = tf.layers.dropout(first_layer_out, dropout_num, seed=seed_num+256, training=training_switch)
    first_maxpool_out = maxpool2d(first_drop_out, pool_size=2)

    second_layer_out = conv2d(first_maxpool_out, weights['second_layer_weights'], biases['second_layer_biases'], normalization_switch)
    second_drop_out = tf.layers.dropout(second_layer_out, dropout_num, seed=seed_num+128, training=training_switch)
    second_maxpool_out = maxpool2d(second_drop_out, pool_size=2)

    third_layer_out = conv2d(second_maxpool_out, weights['third_layer_weights'], biases['third_layer_biases'], normalization_switch)
    third_drop_out = tf.layers.dropout(third_layer_out, dropout_num, seed=seed_num+1024, training=training_switch)
    #third_maxpool_out = maxpool2d(third_drop_out, pool_size=2)
    
    interim_shape = third_drop_out.get_shape().as_list()
    transposed = tf.transpose(third_drop_out, perm=[0, 2, 1, 3])
    reshape_for_rnn = tf.reshape(transposed, [-1, interim_shape[2], interim_shape[1]*interim_shape[3]])
    reshape_for_rnn.set_shape([None, interim_shape[2], interim_shape[1]*interim_shape[3]])
    
    fw_cell = tf.contrib.rnn.GRUCell(hidden_layer)
    bw_cell = tf.contrib.rnn.GRUCell(hidden_layer)
    gru_fw_cell = tf.contrib.rnn.DropoutWrapper(cell=fw_cell, output_keep_prob=rnn_dropout, seed=(seed_num+32))
    gru_bw_cell = tf.contrib.rnn.DropoutWrapper(cell=bw_cell, output_keep_prob=rnn_dropout, seed=(seed_num+64))
    
    gru_output, _ = tf.nn.bidirectional_dynamic_rnn(gru_fw_cell, gru_bw_cell, reshape_for_rnn, dtype=tf.float32)
    gru_output_concatted = tf.concat(gru_output, axis=2)
    
    attention_out = attention(gru_output_concatted, 1)
    
    #interim_shape_gru = tf.shape(gru_output_concatted)
    #gru_flatten = tf.reshape(gru_output_concatted, [-1, interim_shape_gru[1]*interim_shape_gru[2]])
    #gru_dropout = tf.nn.dropout(gru_flatten, rnn_dropout, seed=seed_num+512)
    
    fully_connected_out = tf.add(tf.matmul(attention_out, weights['gru_weights']), biases['gru_biases'])
    fully_connected_activation = tf.nn.leaky_relu(fully_connected_out, 0.01)
    
    prediction = tf.add(tf.matmul(fully_connected_activation, weights['output']), biases['output'])
        
    return prediction

In [ ]:
weights_rnn = {
    'first_layer_weights': tf.Variable(tf.truncated_normal(first_kernel, stddev=0.1, seed=seed_num)),
    'second_layer_weights': tf.Variable(tf.truncated_normal(second_kernel, stddev=0.1, seed=(seed_num+4))),
    'third_layer_weights': tf.Variable(tf.truncated_normal(third_kernel, stddev=0.1,seed=(seed_num+8))),
    'gru_weights': tf.Variable(tf.truncated_normal([2*hidden_layer, dense_layer], stddev=0.1, seed=(seed_num+2))),
    'output': tf.Variable(tf.truncated_normal([dense_layer, output_size], stddev=0.1, seed=(seed_num+16)))
}

biases_rnn = {
    'first_layer_biases': tf.get_variable('first_bias', [first_maps], initializer=tf.constant_initializer(0.1)),
    'second_layer_biases': tf.get_variable('second_bias', [second_maps], initializer=tf.constant_initializer(0.1)),
    'third_layer_biases': tf.get_variable('third_bias', [third_maps], initializer=tf.constant_initializer(0.1)),
    'gru_biases': tf.get_variable('gru_bias', [dense_layer], initializer=tf.constant_initializer(0.1)),
    'output': tf.get_variable('output_bias', [output_size], initializer=tf.constant_initializer(0.1))
}

In [ ]:
def validate_network(session, val_data, val_labels, epoch_step, test_num, test_gender):

    val_generator = batch_generator(val_data, val_labels)
    acc_val_all = []
    loss_val_all = []
    predictions_all = np.asarray([])
    for minibatch, labels, current_index in val_generator:
        loss, acc = sess.run([loss_function, accuracy], feed_dict={X: minibatch, Y: labels, keep_prob: 1.0, keep_rnn_prob: 1.0, keep_fully_prob: 1.0, normalization_switch: False})
        predics = sess.run(tf.argmax(prediction, 1), feed_dict={X: minibatch, Y: labels, keep_prob: 1.0, keep_rnn_prob: 1.0, keep_fully_prob: 1.0, normalization_switch: False})
        predictions_all = np.concatenate((predictions_all, predics))
        acc_val_all.append(acc)
        loss_val_all.append(loss)
    val_acc = np.mean(acc_val_all)
    val_loss = np.mean(loss_val_all)
    validation_confusion = confusion_matrix(predictions_all, [onehot.index(1.0) for onehot in validation_labels])
    validation_recall = np.mean(np.diag(validation_confusion) / np.sum(validation_confusion, axis=1))
    validation_unweighted = np.mean(np.diag(validation_confusion) / np.sum(validation_confusion, axis=0))
    
    print("Validation after epoch #" + str(epoch_step) + ", Validation Loss= "+ "{:.4f}".format(val_loss) + ", Validation Accuracy= " + "{:.3f}".format(val_acc) + ", Unweighted Accuracy= " + "{:.3f}".format(validation_unweighted) + ", Recall= " + " {:.3f}".format(validation_recall))
    print(validation_confusion)

    acc_loss_file_validation = open(checkpoint_folder + "val_loss_logs_session{0}_{1}.txt".format(test_num, test_gender), "a+")
    acc_loss_file_validation.write("{0}, {1}, {2}, {3}, {4}\n".format(epoch_step, val_acc, val_loss, validation_unweighted, validation_recall))
    acc_loss_file_validation.close()
    return val_loss, val_acc, validation_unweighted

In [ ]:
def train_network(session, tr_data, tr_labels, val_data, val_labels, test_num, test_gender):
    
    best_acc = 0
    best_loss = sys.maxsize
    best_train_acc = 0
    current_tolerance = 0
    best_val_acc = 0
    tolerance = 0
    best_val_loss = sys.maxsize
    best_val_unweighted = 0
    
    for epoch_step in range(1, epochs+1):
        batch_gen = batch_generator(tr_data, tr_labels)
        train_acc_all = []
        train_loss_all = []
        for data_minibatch, label_minibatch, current_index in batch_gen:
            session.run(train_trigger, feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: dropout, keep_rnn_prob: recurrent_dropout, keep_fully_prob:final_dropout, normalization_switch: True})
            train_loss, train_acc = session.run([loss_function, accuracy], feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: 1.0, keep_rnn_prob: 1.0, keep_fully_prob: 1.0, normalization_switch: False})
            train_acc_all.append(train_acc)
            train_loss_all.append(train_loss)
        train_acc_whole = np.mean(train_acc_all)
        train_loss_whole = np.mean(train_loss_all)
        acc_loss_file = open(checkpoint_folder + "train_loss_logs_session{0}_{1}.txt".format(test_num,test_gender), "a+")
        acc_loss_file.write("{0}, {1}, {2}\n".format(epoch_step, train_acc_whole, train_loss_whole))
        acc_loss_file.close()
        print("\nTraining accuracy and loss of epoch #" + str(epoch_step) + ": {:.4f}".format(train_acc_whole) + ", {:.4f}".format(train_loss_whole))
        
        val_loss, val_acc, val_unweighted = validate_network(session, val_data, val_labels, epoch_step, test_num, test_gender)
        if best_val_loss > val_loss:
            best_val_loss = val_loss
            saver.save(sess, checkpoint_folder + "loss_session{0}_{1}/best_model_loss.ckpt".format(test_num, test_gender))
            print("Saved model by validation loss {0}".format(val_loss))
            if tolerance != 0:
                tolerance -= 1

        if best_val_unweighted < val_unweighted:
            best_val_unweighted = val_unweighted
            saver.save(sess, checkpoint_folder + "uacc_session{0}_{1}/best_model_uacc.ckpt".format(test_num, test_gender))
            print("Saved model by validation unweighted acc {0}".format(val_unweighted))
            tolerance = 0
        else:
            tolerance += 1
        
        if tolerance >= early_stop:
            break


    print("Optimization Finished!")

In [ ]:
def test_sentence(sess, npz_file, mean, std):
    correct_label = npz_file['labels'][0]
    final_prediction = []
    prediction_add = [0, 0, 0, 0]
    amount = 0
    for spectrogram in npz_file['spectrograms']:
        amount += 1
        to_test = np.asarray(spectrogram)
        to_test = (to_test - mean) / std
        to_test = to_test.reshape(1, 60000)
        
        predic = sess.run(prediction, feed_dict={X: to_test, Y: onehot_encode([correct_label]), keep_prob: 1.0, keep_rnn_prob: 1.0, keep_fully_prob: 1.0, normalization_switch: False})
        prediction_add = prediction_add + predic
    prediction_add = prediction_add[0].tolist()
    prediction_add = [pred/amount for pred in prediction_add]
    final_prediction = prediction_add.index(max(prediction_add))
    encoded = onehot_encode([correct_label])
    correct_pred = encoded[0].index(1.0)
    return final_prediction, correct_pred

In [ ]:
def evaluate_model(session, session_number, session_gender):
    predicted_labels = []
    correct_labels = []
    sentence_amount = 0

    for spectrogram in glob.glob(dataset_folder.format("session{0}".format(session_number))):
        gender = spectrogram.split('\\')[-1].split('_')[2][0]
        if gender == session_gender:
            sentence = np.load(spectrogram)
            final, correct = test_sentence(sess, sentence, dmean, dstd)
            predicted_labels.append(final)
            correct_labels.append(correct)
            sentence_amount += 1
    matrix = confusion_matrix(predicted_labels, correct_labels)
    test_recall = np.mean(np.diag(matrix) / np.sum(matrix, axis=1))
    test_unweighted = np.mean(np.diag(matrix) / np.sum(matrix, axis=0))
    acc = np.sum(np.diag(matrix)) / sentence_amount
    percentage_matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis] * 100
    print("Acc: {0}, UAcc: {1}, Recall: {2}".format(acc, test_unweighted, test_recall))
    print(matrix)
    test_log_file = open(checkpoint_folder + "test_logs_session{0}_{1}.txt".format(session_number, session_gender), "a+")
    test_log_file.write("{0}, {1}, {2}\n".format(acc, test_unweighted, test_recall))
    test_log_file.write("{0}\n".format(matrix))
    test_log_file.close()


In [ ]:
def evaluate_model_results(session, session_number, session_gender):
    predicted_labels = []
    correct_labels = []
    sentence_amount = 0

    for spectrogram in glob.glob(dataset_folder.format("session{0}".format(session_number))):
        gender = spectrogram.split('\\')[-1].split('_')[2][0]
        if gender == session_gender:
            sentence = np.load(spectrogram)
            final, correct = test_sentence(sess, sentence, dmean, dstd)
            predicted_labels.append(final)
            correct_labels.append(correct)
            sentence_amount += 1
    matrix = confusion_matrix(predicted_labels, correct_labels)
    test_recall = np.mean(np.diag(matrix) / np.sum(matrix, axis=1))
    test_unweighted = np.mean(np.diag(matrix) / np.sum(matrix, axis=0))
    acc = np.sum(np.diag(matrix)) / sentence_amount
    percentage_matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis] * 100
    print("Acc: {0}, UAcc: {1}, Recall: {2}".format(acc, test_unweighted, test_recall))
    print(matrix)
    
    return acc, test_unweighted, test_recall, matrix
    

In [ ]:
all_speakers = {1: {'M': (), 'F': ()},
               2: {'M': (), 'F': ()},
               3: {'M': (), 'F': ()},
               4: {'M': (), 'F': ()},
               5: {'M': (), 'F': ()}}

session_string = 'session{0}'

for i in range(1, 6):
    male_spect = []
    female_spect = []
    male_labels = []
    female_labels = []
    formatted = session_string.format(i)
    for spectrogram in glob.glob(dataset_folder.format(formatted)):
        loaded_spec = np.load(spectrogram)
        gender = spectrogram.split('\\')[-1].split('_')[2][0]
        for spect in loaded_spec['spectrograms']:
            if gender == 'M':
                male_spect.append(spect)
            else:
                female_spect.append(spect)
        for label in loaded_spec['labels']:
            if gender == 'M':
                male_labels.append(label)
            else:
                female_labels.append(label)
    all_speakers[i] = {'M': (male_spect, male_labels), 'F': (female_spect, female_labels)}

dmean, dstd = compute_mean_std(all_speakers)

In [ ]:
X = tf.placeholder(tf.float32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, output_size])

keep_prob = tf.placeholder(tf.float32)
keep_rnn_prob = tf.placeholder(tf.float32)
keep_fully_prob = tf.placeholder(tf.float32)
normalization_switch = tf.placeholder(tf.bool)

logits = nn_pipeline_rnn(X, weights_rnn, biases_rnn, keep_prob, keep_rnn_prob, keep_fully_prob, normalization_switch)
prediction = tf.nn.softmax(logits)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

for key in all_speakers:
    train_data, train_labels, class_weights = prepare_train_data(all_speakers, key, dmean, dstd)
    
    weights = tf.reduce_sum(class_weights * Y, axis=1)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y)
    weighted_entropy = cross_entropy * weights
    loss_function = tf.reduce_mean(weighted_entropy)
    cross = tf.reduce_mean(cross_entropy)

    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        train_trigger = optimizer.minimize(loss_function)


    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    for subkey in all_speakers[key]:
        
        test_data, test_labels, validation_data, validation_labels = prepare_validation_test_sets(all_speakers, subkey, key, dmean, dstd)
        
        with tf.device('/gpu:0'):
            sess = tf.Session()
            sess.run(init)

            print('Session initialized with session {0} and test speaker {1}.'.format(key, subkey))
            train_network(sess, train_data, train_labels, validation_data, validation_labels, key, subkey)
            
            saver.restore(sess, checkpoint_folder + "uacc_session{0}_{1}/best_model_uacc.ckpt".format(key, subkey))    
            
            evaluate_model(sess, key, subkey)
    sess.close()


Session initialized with session 1 and test speaker M.

Training accuracy and loss of epoch #1: 0.3557, 1.3548
Validation after epoch #1, Validation Loss= 1.2592, Validation Accuracy= 0.316, Unweighted Accuracy= 0.327, Recall=  0.331
[[ 67  30 108  48]
 [  1   1  17  17]
 [  1   0  51  44]
 [  0   0   2   2]]
Saved model by validation loss 1.2591726779937744
Saved model by validation unweighted acc 0.32695185730508863

Training accuracy and loss of epoch #2: 0.3168, 1.2869
Validation after epoch #2, Validation Loss= 1.2101, Validation Accuracy= 0.379, Unweighted Accuracy= 0.439, Recall=  0.375
[[65 20 74 28]
 [ 2  6 13 16]
 [ 2  5 24 13]
 [ 0  0 67 54]]
Saved model by validation loss 1.210054636001587
Saved model by validation unweighted acc 0.43922382994116604

Training accuracy and loss of epoch #3: 0.4163, 1.2369
Validation after epoch #3, Validation Loss= 1.1655, Validation Accuracy= 0.426, Unweighted Accuracy= 0.473, Recall=  0.418
[[61 18 60 21]
 [ 3  7  8 12]
 [ 5  6 35 13]
 [ 0

Validation after epoch #27, Validation Loss= 0.9433, Validation Accuracy= 0.519, Unweighted Accuracy= 0.555, Recall=  0.496
[[55  8 20  4]
 [ 2 10 21  6]
 [12 13 47  8]
 [ 0  0 90 93]]
Saved model by validation loss 0.9433112144470215

Training accuracy and loss of epoch #28: 0.6739, 0.7585
Validation after epoch #28, Validation Loss= 0.9389, Validation Accuracy= 0.519, Unweighted Accuracy= 0.555, Recall=  0.495
[[55  9 21  4]
 [ 1 10 21  6]
 [13 12 47  8]
 [ 0  0 89 93]]
Saved model by validation loss 0.9389028549194336

Training accuracy and loss of epoch #29: 0.6789, 0.7507
Validation after epoch #29, Validation Loss= 0.9372, Validation Accuracy= 0.519, Unweighted Accuracy= 0.555, Recall=  0.495
[[55  9 20  3]
 [ 1 10 22  6]
 [13 12 47  9]
 [ 0  0 89 93]]
Saved model by validation loss 0.9372427463531494


In [14]:
sess.close()
tf.reset_default_graph()

In [15]:
with tf.device('/gpu:0'):
    sess = tf.Session()

    saver.restore(sess, "./{0}loss/best_model_loss.ckpt".format(checkpoint_folder))    
    train_network(sess)

NotFoundError: FindFirstFile failed for: ./acrnn_topology1loss : The system cannot find the path specified.
; No such process

In [ ]:
sess = tf.Session()

saver.restore(sess, "./{0}uacc/best_model_uacc.ckpt".format(checkpoint_folder))

In [18]:
test_accs = []
test_uaccs = []
test_recalls = []
test_matrices = []

for key in all_speakers:
    for subkey in all_speakers[key]:
        sess = tf.Session()
        saver.restore(sess, checkpoint_folder + "loss_session{0}_{1}/best_model_loss.ckpt".format(key, subkey))        
        tacc, tuacc, trec, tmat = evaluate_model_results(sess, key, subkey)
        test_accs.append(tacc)
        test_uaccs.append(tuacc)
        test_recalls.append(trec)
        test_matrices.append(tmat)

INFO:tensorflow:Restoring parameters from acrnn_topology2_es25loss_session1_M/best_model_loss.ckpt
Acc: 0.7268292682926829, UAcc: 0.7435745060949979, Recall: 0.6742661120673169
[[23  3  9  1]
 [ 0  8  8  0]
 [ 2  4 61  3]
 [ 0  0 26 57]]
INFO:tensorflow:Restoring parameters from acrnn_topology2_es25loss_session1_F/best_model_loss.ckpt
Acc: 0.4976958525345622, UAcc: 0.5743295327768652, Recall: 0.49911473760157976
[[25  4  9  0]
 [ 1  7 15  1]
 [11  7 36  2]
 [ 0  0 59 40]]
INFO:tensorflow:Restoring parameters from acrnn_topology2_es25loss_session2_M/best_model_loss.ckpt
Acc: 0.6153846153846154, UAcc: 0.6686558402007763, Recall: 0.6209779019925857
[[ 9  2  2  0]
 [ 2 19 43  0]
 [ 4  8 67  4]
 [ 0  0 15 33]]
INFO:tensorflow:Restoring parameters from acrnn_topology2_es25loss_session2_F/best_model_loss.ckpt
Acc: 0.5783783783783784, UAcc: 0.6368253968253968, Recall: 0.5306818181818183
[[ 7  7 23  3]
 [ 0  9 11  8]
 [ 0  8 54 15]
 [ 0  1  2 37]]
INFO:tensorflow:Restoring parameters from acrnn

In [22]:
print(np.mean(test_accs), np.mean(test_uaccs), np.mean(test_recalls), (2*(np.mean(test_uaccs)*np.mean(test_recalls))/(np.mean(test_uaccs)+np.mean(test_recalls))))

0.5971253449190548 0.6136593714729905 0.5576317173127283 0.584305528208686


In [20]:
reduce(op.add, test_matrices)

array([[232,  59, 155,  29],
       [ 20, 110, 218,  18],
       [ 34, 101, 545,  89],
       [  3,  14, 181, 472]], dtype=int64)

In [ ]:
def get_confusion_matrix(session, data, labels):
    predictions = session.run(tf.argmax(prediction, 1), feed_dict={X:data, Y:labels, keep_prob:1.0, keep_rnn_prob: 1.0, normalization_switch: False})
    true_labels = [onehot.index(1.0) for onehot in labels]
    return confusion_matrix(predictions, true_labels)

def get_final_results(session, data, labels):
    final_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    overall_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    predictions = session.run(correct_prediction, feed_dict={X:data, Y:labels, keep_prob:1.0,  keep_rnn_prob: 1.0,  normalization_switch: False})
    coupled = list(zip(predictions, onehot_decode(labels)))
    
    for couple in coupled:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    
    return final_dict, overall_dict

def get_final_results_glued(session, labels, correct):
    final_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    overall_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    coupled = list(zip(correct, onehot_decode(labels)))
    
    for couple in coupled:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    
    return final_dict, overall_dict

In [ ]:
final_preds = np.asarray([])
final_correct_preds = np.asarray([])
final_probs = np.empty((0, 4))
final_probs_softmax = np.empty((0, 4))


midpoint = math.ceil(len(test_labels)/2)
for i in range(0, 2):
    predictions_test = sess.run(correct_prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, keep_rnn_prob: 1.0, normalization_switch: False})
    probabilities_test = sess.run(prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, keep_rnn_prob: 1.0, normalization_switch: False})
    predictions_test_confusion = sess.run(tf.argmax(prediction, 1), feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, keep_rnn_prob: 1.0, normalization_switch: False})
    predictions_test_softmax = sess.run(prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, keep_rnn_prob: 1.0, normalization_switch: False})

    final_correct_preds = np.concatenate((final_correct_preds, predictions_test))
    final_probs = np.concatenate((final_probs, probabilities_test))
    final_preds = np.concatenate((final_preds, predictions_test_confusion))
    final_probs_softmax = np.concatenate((final_probs_softmax, predictions_test_softmax))

In [ ]:
confusion_matrix(final_preds, [onehot.index(1.0) for onehot in test_labels])

In [ ]:
conf = confusion_matrix(final_preds, [onehot.index(1.0) for onehot in test_labels])
np.mean(np.diag(conf) / np.sum(conf, axis=0))

In [ ]:
final_dict_test, overall_dict_test = get_final_results_glued(sess, test_labels, final_correct_preds)

In [ ]:
print(final_dict_test)
print(overall_dict_test)

In [ ]:
print(unweighted_accuracy(final_dict_test, overall_dict_test))
print(weighted_accuracy(final_dict_test, overall_dict_test))

In [ ]:
print(precision_recall_fscore_support(final_preds, [onehot.index(1.0) for onehot in test_labels], average='macro')[0:2])

In [ ]:
final_probs_softmax

In [ ]:
matrix

In [ ]:
val_losses = list(map(lambda line: float(line.split(", ")[2]), open(checkpoint_folder + "val_loss_logs.txt")))
val_accs = list(map(lambda line: float(line.split(", ")[1]), open(checkpoint_folder + "val_loss_logs.txt")))
val_uaccs = list(map(lambda line: float(line.split(", ")[3]), open(checkpoint_folder + "val_loss_logs.txt")))

In [ ]:
train_losses = list(map(lambda line: float(line.split(", ")[2]), open(checkpoint_folder + "train_loss_logs.txt")))
train_accs = list(map(lambda line: float(line.split(", ")[1]), open(checkpoint_folder + "train_loss_logs.txt")))
epochs = list(map(lambda line: int(line.split(", ")[0]), open(checkpoint_folder + "train_loss_logs.txt")))

In [ ]:
import matplotlib.pyplot as plt

plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(epochs, train_losses, epochs, val_losses)
plt.legend(["Train", "Validation"])
plt.savefig("./graphs_validation/{0}loss.png".format(checkpoint_folder))

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(epochs, train_accs, epochs, val_accs)
plt.legend(["Train", "Validation"])
plt.savefig("./graphs_validation/{0}acc.png".format(checkpoint_folder))

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(epochs, val_uaccs)
plt.legend(['Validation'])
plt.savefig("./graphs_validation/{0}uacc.png".format(checkpoint_folder))